In [1]:
import pandas as pd
import numpy as np
import operator
import ast
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
import string
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
import spacy
import re

In [2]:
data = pd.read_csv('combined-newsqa-data-v1.csv')
print(data.columns)
print(type(data['is_question_bad'][0]))

# remove Q/A pairs that are invalid or missing
#data = data[(data.is_question_bad=='0.0') & (data.is_answer_absent=='0.0')]
data.head()

Index(['story_id', 'question', 'answer_char_ranges', 'is_answer_absent',
       'is_question_bad', 'validated_answers', 'story_text'],
      dtype='object')
<class 'str'>


,story_id,question,answer_char_ranges,is_answer_absent,is_question_bad,validated_answers,story_text
0,./cnn/stories/42d01e187213e86f5fe617fe32e716ff...,What was the amount of children murdered?,294:297|None|None,0.0,0.0,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort..."
1,./cnn/stories/c48228a52f26aca65c31fad273e66164...,Where was one employee killed?,34:60|1610:1618|34:60,0.0,0.0,NaN,(CNN) -- Fighting in the volatile Sudanese reg...
2,./cnn/stories/c65ed85800e4535f4bbbfa2c34d7d963...,who did say South Africa did not issue a visa ...,103:127|114:127|839:853,0.0,0.0,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...
3,./cnn/stories/0cf66b646e9b32076513c050edf32a79...,How many years old was the businessman?,538:550|538:550,0.0,0.0,NaN,(CNN) -- England international footballer Ste...
4,./cnn/stories/13012604e3203c18df09289dfedd14cd...,What frightened the families?,690:742|688:791|630:646,0.0,0.0,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi..."


In [3]:
print(data['is_question_bad'][0])

0.0


In [4]:
# 119,633 Q/A's , 12088 articles
print(len(data))
print(len(data['story_text'].unique()))


119633
12088


In [5]:
first_doc = data['story_text'][0]
first_doc

'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors" by the Indian media.\n\n\n\nPandher was not named a main suspect by 

In [6]:
first_doc[294:297]

'19 '

In [7]:
second_doc = data['story_text'][2]
second_doc

'Johannesburg (CNN) -- Miffed by a visa delay that led the Dalai Lama to cancel a trip to South Africa, Archbishop Desmond Tutu lashed out at his government Tuesday, saying it had acted worse than apartheid regimes and had forgotten all that the nation stood for.\n\n\n\n\n\n"When we used to apply for passports under the apartheid government, we never knew until the last moment what their decision was," Tutu said at a news conference. "Our government is worse than the apartheid government because at least you were expecting it from the apartheid government.\n\n\n\n\n\n"I have to say that I can\'t believe this. I really can\'t believe this," Tutu said. "You have to wake me up and tell me this is actually happening here."\n\n\n\n\n\nThe Dalai Lama scrapped his planned trip to South Africa this week after the nation failed to issue him a visa in time, his spokesman said.\n\n\n\n\n\nVisa applications for him and his entourage were submitted to the South African High Commission in New Delhi,

In [8]:
print(second_doc[103:127])
print(second_doc[114:127])
print(second_doc[839:853])

Archbishop Desmond Tutu 
Desmond Tutu 
his spokesman 


Answers given by different human reviewers are in the answer_char_ranges column and the validated_answers column. These values are string index ranges within the document that represent the answer. For each row, we need to determine which character range is the best answer to use and extract it. There are a couple ways we could approach this problem, the first is to have a model that takes the question(str), the story(str), spits out an answer(str).

or we could have it spit back out the string indexes themselves. 

In [70]:
# Remove uneeded columns
df = data[['question','validated_answers','story_text']]
df = df.dropna()
df = df.iloc[0:100] # reduce size for development purposes

df.head()

,question,validated_answers,story_text
0,What was the amount of children murdered?,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort..."
2,who did say South Africa did not issue a visa ...,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...
4,What frightened the families?,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi..."
6,Who is hiring?,"{""301:324"": 2}",CNN affiliates report on where job seekers are...
8,Iran criticizes who?,"{""63:97"": 2}","TEHRAN, Iran (CNN) -- Iran's parliament speake..."


In [72]:
# loops through the rows and prints the question along with the first answer given
start_truth = []
end_truth = []
for i, row in df.iterrows():
    try:
        answers = ast.literal_eval(row['validated_answers'])
        sorted_ans = sorted(answers.items(), key=operator.itemgetter(1), reverse=True)
        #print(sorted_ans)
        start, end = sorted_ans[0][0].split(':')
        start_truth.append(int(start))
        end_truth.append(int(end))
    except ValueError:
        start_truth.append(np.nan)
        end_truth.append(np.nan)
        pass
df['start_truth'] = start_truth
df['end_truth'] = end_truth
df = df.dropna()
print(len(df))
df.head()

80


,question,validated_answers,story_text,start_truth,end_truth
0,What was the amount of children murdered?,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort...",294.0,297.0
2,who did say South Africa did not issue a visa ...,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...,103.0,127.0
4,What frightened the families?,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi...",688.0,791.0
6,Who is hiring?,"{""301:324"": 2}",CNN affiliates report on where job seekers are...,301.0,324.0
8,Iran criticizes who?,"{""63:97"": 2}","TEHRAN, Iran (CNN) -- Iran's parliament speake...",63.0,97.0


In [73]:
final = df.drop(columns=['validated_answers'])
final.head()

,question,story_text,start_truth,end_truth
0,What was the amount of children murdered?,"NEW DELHI, India (CNN) -- A high court in nort...",294.0,297.0
2,who did say South Africa did not issue a visa ...,Johannesburg (CNN) -- Miffed by a visa delay t...,103.0,127.0
4,What frightened the families?,"BAGHDAD, Iraq (CNN) -- At least 6,000 Christi...",688.0,791.0
6,Who is hiring?,CNN affiliates report on where job seekers are...,301.0,324.0
8,Iran criticizes who?,"TEHRAN, Iran (CNN) -- Iran's parliament speake...",63.0,97.0


In [74]:
full_text = ''
m = 0
for col in ['question','story_text']:
    for text in final[col]:
        m = max(m, len(text))
        full_text += text.lower()
    print(m)
    m=0

# get the set of all characters
characters = tuple(set(full_text))
characters[:5]

68
7361


('u', '(', ';', 'd', 't')

In [79]:
# use enumeration to give the characters integer values
int2char = dict(enumerate(characters))

# create the look up dictionary from characters to the assigned integers
char2int = {char: index for index, char in int2char.items()}
print(char2int)


{'u': 0, '(': 1, ';': 2, 'd': 3, 't': 4, '.': 5, 'o': 6, 'é': 7, '2': 8, ',': 9, '0': 10, 'p': 11, '_': 12, 'â': 13, 'ñ': 14, '5': 15, 'ã': 16, 'v': 17, '7': 18, ':': 19, 'j': 20, 's': 21, '[': 22, 'g': 23, ')': 24, 'a': 25, 'h': 26, '&': 27, '8': 28, 'f': 29, 'y': 30, '\n': 31, 'b': 32, 'w': 33, ' ': 34, '6': 35, "'": 36, '*': 37, 'ë': 38, 'ú': 39, 'l': 40, '»': 41, '©': 42, 'e': 43, '9': 44, 'r': 45, '3': 46, '•': 47, '4': 48, 'n': 49, '?': 50, '!': 51, '+': 52, ']': 53, 'i': 54, 'c': 55, '-': 56, '/': 57, 'q': 58, 'k': 59, 'x': 60, '"': 61, '1': 62, 'm': 63, 'z': 64, '$': 65}


In [80]:
q_max = 68
story_max = 7361

In [90]:
padding_start = False

for i, row in final.iterrows():
    for col, N in zip(['question','story_text'], [q_max, story_max]):
        enc = np.zeros(N, dtype=np.int32)
        
        row[col] = np.array([char2int[char] for char in row[col].lower()])
        l = min(N, len(row[col]))
        if padding_start:
            enc[:l] = row[col][:l]
        else:
            enc[N-l:] = row[col][:l]
        final[col][i] = enc
final.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,question,story_text,start_truth,end_truth
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",294.0,297.0
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 33,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",103.0,127.0
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",688.0,791.0
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",301.0,324.0
8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",63.0,97.0


In [91]:
#final2 = final.reset_index().drop(columns=['index'])
#print(final2.index) 
#final2.head()

RangeIndex(start=0, stop=80, step=1)


,question,story_text,start_truth,end_truth
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",294.0,297.0
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 33,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",103.0,127.0
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",688.0,791.0
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",301.0,324.0
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",63.0,97.0


## Train/Test Split

In [93]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final[['question', 'story_text']], 
                                                    final[['start_truth','end_truth']], 
                                                    test_size=0.3, 
                                                    random_state=42)

In [94]:
print(X_train.index)
X_train.head()

Int64Index([ 85, 170, 118, 148,  31, 112, 152, 226,  13, 138, 146,  39, 187,
             54, 120,  22, 223,   6,  35, 109,  15, 183,  11,  94, 213, 159,
            126, 225,  26,  66, 116,  63, 132,  48, 119, 224, 154,  18,  80,
            220, 160, 174, 196, 105,  69,   2, 144,  42,   4,  47, 219,  40,
            161, 211,  25, 139],
           dtype='int64')


,question,story_text
85,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
170,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
118,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
148,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
31,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [95]:
y_train.head()

,start_truth,end_truth
85,2601.0,2606.0
170,1250.0,1308.0
118,64.0,98.0
148,1765.0,1777.0
31,1873.0,1907.0


In [96]:
class NewsQADataset(Dataset):
    def __init__(self, X, y, N=400, padding_start=False):
        self.y = y.values
        self.question = X['question'].values
        self.story_text =X['story_text'].values
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.question[idx], self.story_text[idx], self.y[idx]

In [97]:
train_ds = NewsQADataset(X_train, y_train)
test_ds = NewsQADataset(X_test, y_test)

In [98]:

train_ds[0]

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 54, 49, 34,
        33, 26, 25,  4, 34, 30, 43, 25, 45, 34,  3, 54,  3, 34, 32, 63, 33,
        34, 43, 49,  4, 43, 45, 34,  4, 26, 43, 34, 21, 11,  6, 45,  4, 50],
       dtype=int32),
 array([ 0,  0,  0, ..., 25,  3,  5], dtype=int32),
 array([2601., 2606.]))

In [99]:
batch_size = 8
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size)

In [102]:
x1, x2, y = next(iter(train_dl))
print(x1.shape, x2.shape, y.shape)
x1, x2, y

torch.Size([8, 68]) torch.Size([8, 7361]) torch.Size([8, 2])


(tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0, 33, 26,  6, 34, 54, 21, 34, 26, 43, 49,  3, 43, 45, 21,  6, 49,
          34, 25, 34, 21, 30, 63, 32,  6, 40, 34, 29,  6, 45, 50],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0, 33, 26, 43, 45, 43, 34,  3, 54,  3, 34,  4, 26, 43, 34,
          29, 40, 54, 23, 26,  4, 34, 25, 45, 45, 54, 17, 43, 50],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0, 33, 26, 25,  4, 34, 23, 45,  6,  0, 11, 34, 54, 21, 34, 32,
          43, 54, 49, 23, 34,  4, 25, 45, 23, 43,  4, 43,  3, 50],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

In [153]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharRNN, self).__init__()

        self.hidden_size = hidden_size
        self.linear_i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.linear_h2o = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        combined = torch.cat((x, hidden), 1)
        hidden = torch.tanh(self.linear_i2h(combined))
        output = self.linear_h2o(hidden)
        return output, hidden

    def initHidden(self, bash_size):
        return torch.zeros(bash_size, self.hidden_size)

In [164]:
vocab_size = 66
hidden_size = 100
n_classes = 2
model = CharRNN(vocab_size, hidden_size, n_classes)

In [165]:
def get_optimizer(model, lr = 0.01, wd = 0.00001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

In [170]:
def train(model, optim, train_dl):
    model.train()
    total = 0
    sum_loss = 0
    for x1, x2, y in train_dl:
         # all zeros
        loss = 0
        x = torch.cat((x1, x2), 1)
        print(x)
        x = x.float()
        y = y.long()
        batch = x.shape[0]
        h = model.initHidden(batch)
        print(h.shape)
        print(x.shape)
        
        for t in range(x.shape[1]):
            print(x[:,t])
            out, h = model(x[:,t], h)
        
        loss = F.cross_entropy(out, y)
        optim.zero_grad()
        loss.backward()
        #loss.backward(retain_graph=True)
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
    return sum_loss/total

In [171]:
optim = get_optimizer(model)

loss = train(model, optim, train_dl)
        

tensor([[ 0,  0,  0,  ..., 25,  3,  5],
        [ 0,  0,  0,  ..., 45,  4,  5],
        [ 0,  0,  0,  ...,  6, 49,  5],
        ...,
        [ 0,  0,  0,  ..., 25, 21,  5],
        [ 0,  0,  0,  ..., 45,  4,  5],
        [ 0,  0,  0,  ..., 11, 21,  5]], dtype=torch.int32)
torch.Size([8, 100])
torch.Size([8, 7429])
tensor([0., 0., 0., 0., 0., 0., 0., 0.])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [120]:
train_loop(model, 0.01, train_dl, test_dl, epochs=20)

UnboundLocalError: local variable 'x' referenced before assignment